In [ ]:
# 导入必要的包以及定义函数
import os
import glob
import base64
import time
import requests
import json
import trimesh
import numpy as np

# 以base64上传到服务器

def load_mesh_b64(p):
    with open(p, 'rb') as f:
        data = f.read()
    return base64.b64encode(data).decode()

def b64_to_mesh(d):
    with open("_tmp.stl", 'wb') as f:
        f.write(base64.b64decode(d))
    M = trimesh.load_mesh("_tmp.stl", process=False)
    os.remove("_tmp.stl")
    return M

def _create_colors():
    # 20 high contrast colors
    colors = [[230, 25, 75,255],[60, 180, 75,255],[255, 225, 25,255],\
            [0, 130, 200,255],[245, 130, 48,255],[145, 30, 180,255],[70, 240, 240,255],\
            [240, 50, 230,255],[210, 245, 60,255],[250, 190, 190,255],[0, 128, 128,255],\
            [230, 190, 255,255],[170, 110, 40,255],[255, 250, 200,255],[128, 0, 0,255],\
            [170, 255, 195,255],[128, 128, 0, 255]]
    #np.random.shuffle(colors)
    # gum color
    colors = [[255,255,255,255]] + colors
    return colors

def colored_mesh(mesh, label):
    COLORS = _create_colors()
    mcopy = mesh.copy()
    for i, l in enumerate(np.unique(label)):
        mcopy.visual.face_colors[np.where(label == l)[0]] = COLORS[i % 18]
    return mcopy

# 定义调用规则

请根据您从我方获取的信息修改以下代码块

In [ ]:
# 朝厚服务请求地址，随api文档发送
base_url = "<服务请求地址>"

# 必须传入鉴权 Header。请保护好TOKEN!!! 如果泄露请立即联系我们重置，所有使用该TOKEN的任务都会向您的账户计费
zh_token = "<贵司服务Token, 随合同发送>" # 调用所有的API都必须传入token用作鉴权

# 以下请求数据以分牙+牙轴任务举例，依据您与我方签订的合约，您可能没有权限调用该样例API。
payload = json.dumps({
  "spec_group": "mesh-processing", # 调用的工作流组， 随API文档发送
  "spec_name": "oral-seg-and-axis", # 调用的工作流名称， 随API文档发送, 
  "spec_version": "1.0-snapshot", # 调用的工作流版本，随API文档发送
  "user_group": "APIClient", # 用户组，一般为 APIClient
  "user_id": "<您的user_id>", # 用户ID, 随合同发送
  "input_data": { # 输入数据，此处依照不同的任务字典内容会有不同，具体参照API文档，该处以分牙+牙轴举例
    "mesh": load_mesh_b64('u.stl'), # load_mesh_b64(stl文件路径)，此处需要您提供一个半颌口扫模型
    "jaw_type": 'Upper' # 上颌是Upper, 下颌是Lower
  },
})

# 提交请求

使用 `/run` POST方法提交请求

In [ ]:
headers = {
  "Content-Type": "application/json",
  "X-ZH-TOKEN": zh_token
}

url = base_url + '/run'

response = requests.request("POST", url, headers=headers, data=payload)
response.raise_for_status()
create_result = response.json()
run_id = create_result['run_id']

# 打印任务号
print(run_id)

# 等待请求完成

使用轮询API等待请求完成，API地址为 `/run/{run_id}`

In [ ]:
# 第二步： 轮询状态
url = base_url + f"/run/{run_id}"

start_time = time.time()
while time.time()-start_time < 180: # 最多等3分钟
    time.sleep(3) # 轮询间隔
    response = requests.request("GET", url, headers=headers)
    result = response.json()
    if result['completed'] or result['failed']:
        break
    
if not result['completed']:
    if result['failed']:
        raise ValueError("API运行错误，原因： " + str(result['reason_public']))
    raise TimeoutError("API超时")

print("API运行时间： {}s".format(time.time()-start_time))

# 获取请求结果

使用 `/data/{run_id}` 获得该工作流所有输出数据，使用 `/data/{run_id}/{key}` 获取某一项数据

In [ ]:
url = base_url + f"/data/{run_id}"
response = requests.request("GET", url, headers=headers)
result = response.json()

# 样例API输出结果展示

以下为该分牙+牙轴样例API的输出结果展示。该部分之上代码对所有API通用，以下部分仅针对分牙+牙轴输出处理及展示

In [ ]:
print('牙号列表：', np.unique(result['seg_labels']))
colored_mesh(b64_to_mesh(result['mesh']), result['seg_labels']).show()

In [ ]:
# 将所有牙号转到局部坐标系
def _get_vid_new_faces_by_fid(mesh, inds):
    old_faces = mesh.faces[inds].reshape(-1)
    vid = np.unique(old_faces)
    vid_map = dict(zip(vid, np.arange(len(vid))))
    faces = np.array([vid_map[x] for x in old_faces]).reshape(-1, 3)
    return vid, faces

def get_mesh_by_id(mesh, inds):
    vid, faces = _get_vid_new_faces_by_fid(mesh, inds)
    return trimesh.Trimesh(mesh.vertices[vid], faces)
def split_tooth(mesh, label):
    splitted = {}
    adj_dict = {}
    unique_labels = np.unique(label)

    adj_lab = np.zeros((len(label),))

    for x, y in mesh.face_adjacency:
        if label[x] != label[y]:
            if label[x] == 0 or label[y] == 0:
                adj_lab[x] = max(1, adj_lab[x])
                adj_lab[y] = max(1, adj_lab[y])
            else:
                adj_lab[x] = 2
                adj_lab[y] = 2

    for fdi in unique_labels:
        if fdi == 0:
            continue
        idx = np.where(label == fdi)[0]
        splitted[str(int(fdi))] = get_mesh_by_id(mesh, idx)
        adj_dict[str(int(fdi))] = adj_lab[idx]
    return splitted

all_t = None
for k, t in split_tooth(b64_to_mesh(result['mesh']), result['seg_labels']).items():
    all_t += t.apply_transform(result['axis'][k])

all_t.show()